In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

class DemoSVM:
    def __init__(self):
        self.modelos = {}
        self.scaler = StandardScaler()
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
    
    def generar_datos_ejemplo(self):
        """
        Genera diferentes tipos de datos para demostrar SVM
        """
        print("=== GENERANDO DATOS DE EJEMPLO ===")
        
        # Dataset 1: Datos linealmente separables
        np.random.seed(42)
        n_samples = 100
        
        # Clase 0: círculo pequeño en el centro
        clase_0 = np.random.randn(n_samples//2, 2) * 0.5 + np.array([1, 1])
        
        # Clase 1: círculo más grande alrededor
        angulos = np.random.uniform(0, 2*np.pi, n_samples//2)
        radios = np.random.uniform(2.5, 3.5, n_samples//2)
        clase_1_x = radios * np.cos(angulos)
        clase_1_y = radios * np.sin(angulos)
        clase_1 = np.column_stack((clase_1_x, clase_1_y))
        
        # Combinar datos
        X = np.vstack((clase_0, clase_1))
        y = np.hstack((np.zeros(n_samples//2), np.ones(n_samples//2)))
        
        return X, y
    
    def cargar_dataset_iris(self):
        """
        Carga el famoso dataset Iris para demostración
        """
        print("=== CARGANDO DATASET IRIS ===")
        iris = datasets.load_iris()
        
        # Usar solo 2 características para visualización fácil
        X = iris.data[:, :2]  # longitud y ancho del sépalo
        y = iris.target
        
        # Usar solo 2 clases para simplificar
        mask = y != 2  # Excluir clase virginica
        X = X[mask]
        y = y[mask]
        
        print(f"Características: {iris.feature_names[:2]}")
        print(f"Clases: {iris.target_names[:2]}")
        print(f"Muestras: {len(X)}")
        
        return X, y
    
    def preparar_datos(self, X, y):
        """
        Divide y normaliza los datos
        """
        print("\n=== PREPARANDO DATOS ===")
        
        # Dividir en entrenamiento y prueba
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )
        
        # Normalizar los datos (importante para SVM)
        self.X_train_scaled = self.scaler.fit_transform(self.X_train)
        self.X_test_scaled = self.scaler.transform(self.X_test)
        
        print(f"Entrenamiento: {len(self.X_train)} muestras")
        print(f"Prueba: {len(self.X_test)} muestras")
    
    def entrenar_modelos(self):
        """
        Entrena diferentes tipos de SVM
        """
        print("\n=== ENTRENANDO MODELOS SVM ===")
        
        # 1. SVM Lineal
        print("1. Entrenando SVM Lineal...")
        self.modelos['lineal'] = svm.SVC(kernel='linear', C=1.0, random_state=42)
        self.modelos['lineal'].fit(self.X_train_scaled, self.y_train)
        
        # 2. SVM con kernel RBF (Gaussiano)
        print("2. Entrenando SVM con kernel RBF...")
        self.modelos['rbf'] = svm.SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
        self.modelos['rbf'].fit(self.X_train_scaled, self.y_train)
        
        # 3. SVM con kernel Polinomial
        print("3. Entrenando SVM con kernel Polinomial...")
        self.modelos['poly'] = svm.SVC(kernel='poly', degree=3, C=1.0, random_state=42)
        self.modelos['poly'].fit(self.X_train_scaled, self.y_train)
        
        print("✅ Todos los modelos entrenados exitosamente")
    
    def evaluar_modelos(self):
        """
        Evalúa el rendimiento de cada modelo
        """
        print("\n=== EVALUANDO MODELOS ===")
        
        resultados = {}
        
        for nombre, modelo in self.modelos.items():
            # Predicciones
            y_pred = modelo.predict(self.X_test_scaled)
            
            # Calcular métricas
            precision = accuracy_score(self.y_test, y_pred)
            resultados[nombre] = {
                'precision': precision,
                'predicciones': y_pred,
                'modelo': modelo
            }
            
            print(f"\n--- {nombre.upper()} ---")
            print(f"Precisión: {precision:.3f} ({precision*100:.1f}%)")
            print(f"Vectores de soporte: {len(modelo.support_)}")
            
            # Reporte detallado
            print("Reporte de clasificación:")
            print(classification_report(self.y_test, y_pred, target_names=['Clase 0', 'Clase 1']))
        
        return resultados
    
    def visualizar_resultados(self, resultados):
        """
        Visualiza los resultados de los diferentes modelos SVM
        """
        print("\n=== GENERANDO VISUALIZACIONES ===")
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Comparación de Modelos SVM', fontsize=16, fontweight='bold')
        
        # Colores para las clases
        colores = ['red', 'blue']
        
        # Gráfico 1: Datos originales
        ax = axes[0, 0]
        for i in range(2):
            mask = self.y_train == i
            ax.scatter(self.X_train[mask, 0], self.X_train[mask, 1], 
                      c=colores[i], label=f'Clase {i}', alpha=0.7, s=50)
        
        ax.set_title('Datos de Entrenamiento')
        ax.set_xlabel('Característica 1')
        ax.set_ylabel('Característica 2')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # Gráficos 2-4: Resultados de cada modelo
        posiciones = [(0, 1), (1, 0), (1, 1)]
        nombres_modelos = ['lineal', 'rbf', 'poly']
        titulos = ['SVM Lineal', 'SVM RBF', 'SVM Polinomial']
        
        for idx, (nombre, titulo) in enumerate(zip(nombres_modelos, titulos)):
            ax = axes[posiciones[idx]]
            modelo = resultados[nombre]['modelo']
            precision = resultados[nombre]['precision']
            
            # Datos de entrenamiento
            for i in range(2):
                mask = self.y_train == i
                ax.scatter(self.X_train[mask, 0], self.X_train[mask, 1], 
                          c=colores[i], alpha=0.6, s=30)
            
            # Vectores de soporte
            vectores_soporte = self.X_train[modelo.support_]
            ax.scatter(vectores_soporte[:, 0], vectores_soporte[:, 1], 
                      s=100, facecolors='none', edgecolors='black', linewidths=2)
            
            # Crear malla para mostrar la frontera de decisión
            h = 0.1
            x_min, x_max = self.X_train[:, 0].min() - 1, self.X_train[:, 0].max() + 1
            y_min, y_max = self.X_train[:, 1].min() - 1, self.X_train[:, 1].max() + 1
            xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                                np.arange(y_min, y_max, h))
            
            # Predecir en la malla
            malla_puntos = np.c_[xx.ravel(), yy.ravel()]
            malla_puntos_scaled = self.scaler.transform(malla_puntos)
            Z = modelo.predict(malla_puntos_scaled)
            Z = Z.reshape(xx.shape)
            
            # Mostrar la frontera de decisión
            ax.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.RdYlBu)
            ax.contour(xx, yy, Z, colors='black', linewidths=1, alpha=0.5)
            
            ax.set_title(f'{titulo}\nPrecisión: {precision:.3f}')
            ax.set_xlabel('Característica 1')
            ax.set_ylabel('Característica 2')
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Matriz de confusión para el mejor modelo
        mejor_modelo = max(resultados.items(), key=lambda x: x[1]['precision'])
        nombre_mejor = mejor_modelo[0]
        info_mejor = mejor_modelo[1]
        
        plt.figure(figsize=(8, 6))
        cm = confusion_matrix(self.y_test, info_mejor['predicciones'])
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=['Clase 0', 'Clase 1'],
                   yticklabels=['Clase 0', 'Clase 1'])
        plt.title(f'Matriz de Confusión - Mejor Modelo ({nombre_mejor.upper()})')
        plt.ylabel('Verdadero')
        plt.xlabel('Predicción')
        plt.show()
    
    def ejemplo_prediccion_individual(self):
        """
        Demuestra cómo hacer predicciones individuales
        """
        print("\n=== EJEMPLO DE PREDICCIÓN INDIVIDUAL ===")
        
        # Tomar el mejor modelo
        mejor_modelo = max(self.modelos.items(), 
                          key=lambda x: accuracy_score(self.y_test, x[1].predict(self.X_test_scaled)))
        nombre_mejor, modelo_mejor = mejor_modelo
        
        print(f"Usando el mejor modelo: {nombre_mejor.upper()}")
        
        # Crear un punto nuevo para predecir
        punto_nuevo = np.array([[4.5, 2.8]])  # Ejemplo de características
        punto_nuevo_scaled = self.scaler.transform(punto_nuevo)
        
        # Hacer predicción
        prediccion = modelo_mejor.predict(punto_nuevo_scaled)[0]
        
        # Si el modelo soporta probabilidades
        try:
            modelo_prob = svm.SVC(kernel=modelo_mejor.kernel, C=1.0, probability=True, random_state=42)
            modelo_prob.fit(self.X_train_scaled, self.y_train)
            probabilidades = modelo_prob.predict_proba(punto_nuevo_scaled)[0]
            
            print(f"\nPunto nuevo: {punto_nuevo[0]}")
            print(f"Predicción: Clase {int(prediccion)}")
            print(f"Probabilidades:")
            print(f"  - Clase 0: {probabilidades[0]:.3f} ({probabilidades[0]*100:.1f}%)")
            print(f"  - Clase 1: {probabilidades[1]:.3f} ({probabilidades[1]*100:.1f}%)")
        except:
            print(f"\nPunto nuevo: {punto_nuevo[0]}")
            print(f"Predicción: Clase {int(prediccion)}")
    
    def ejecutar_demo_completa(self, usar_iris=True):
        """
        Ejecuta la demostración completa del SVM
        """
        print("🚀 INICIANDO DEMOSTRACIÓN DE SUPPORT VECTOR MACHINE (SVM)")
        print("=" * 60)
        
        # Cargar datos
        if usar_iris:
            X, y = self.cargar_dataset_iris()
        else:
            X, y = self.generar_datos_ejemplo()
        
        # Preparar datos
        self.preparar_datos(X, y)
        
        # Entrenar modelos
        self.entrenar_modelos()
        
        # Evaluar modelos
        resultados = self.evaluar_modelos()
        
        # Visualizar resultados
        self.visualizar_resultados(resultados)
        
        # Ejemplo de predicción
        self.ejemplo_prediccion_individual()
        
        # Resumen final
        print("\n" + "=" * 60)
        print("📊 RESUMEN FINAL")
        print("=" * 60)
        
        mejor_precision = 0
        mejor_modelo_nombre = ""
        
        for nombre, info in resultados.items():
            precision = info['precision']
            print(f"{nombre.upper():<12}: {precision:.3f} ({precision*100:.1f}%)")
            if precision > mejor_precision:
                mejor_precision = precision
                mejor_modelo_nombre = nombre
        
        print(f"\n🏆 MEJOR MODELO: {mejor_modelo_nombre.upper()}")
        print(f"   Precisión: {mejor_precision:.3f} ({mejor_precision*100:.1f}%)")
        
        print("\n✅ Demostración completada exitosamente!")

# Función principal
def main():
    print("¡Bienvenido al Demo de Support Vector Machine!")
    print("\nElige el tipo de datos:")
    print("1. Dataset Iris (recomendado)")
    print("2. Datos generados artificialmente")
    
    while True:
        try:
            opcion = input("\nIngresa tu opción (1 o 2): ").strip()
            if opcion == '1':
                usar_iris = True
                break
            elif opcion == '2':
                usar_iris = False
                break
            else:
                print("Por favor, ingresa 1 o 2")
        except:
            print("Entrada inválida. Por favor, ingresa 1 o 2")
    
    # Crear y ejecutar la demostración
    demo = DemoSVM()
    demo.ejecutar_demo_completa(usar_iris=usar_iris)

# Ejecutar el programa
if __name__ == "__main__":
    main()

¡Bienvenido al Demo de Support Vector Machine!

Elige el tipo de datos:
1. Dataset Iris (recomendado)
2. Datos generados artificialmente
